In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")

### Pipeline psuedo code
    n = look back window
    k = number of PCs to keep

    for each time point t:
        p = number of stocks in investable universe at time t
        Define an n x p feature matrix X (lagged returns)

        Perform PCA on X
        Keep the first k PCs in an n x k matrix Z

    for each stock s in the investable universe at time t:
        Define an n x 1 outcome vector y (future returns of stock s)
        Perform a linear regression of y on Z
        Predict y for stock s at time t+1

## Data Preparation

In [ ]:
returns = pd.read_pickle("./Data/returns.pkl")
returns = returns.iloc[1:]

In [ ]:
drop_columns = []

for col in returns.columns:
    if returns[col].isnull().all() == True:
        drop_columns.append(col)
        
returns.drop(columns=drop_columns, inplace=True)

In [ ]:
returns_monthly = pd.read_pickle("./Data/returns-monthly.pkl")
returns_monthly

## Pipeline

In [ ]:
def get_investable(t, n):
    """ Find stocks in investable universe at time t+1
    (stocks in the S&P500 that have prices recorded for the last n days)"""
    
    df_investable = returns.copy(deep = True).sort_index(ascending = False)
    
    #add 1 date to get the test features in investable
    t = t + pd.DateOffset(1)
    n += 1
    
    #if t is now a non-trading day, advance until we reach a valid trading day
    while t not in df_investable.index:
        t = t + pd.DateOffset(1)
    
    t_index = df_investable.index.get_loc(t)
    
    #take n_rows worth of data upto time specified
    df_investable = df_investable.iloc[t_index:t_index + n]
    
    #find all stocks that exist in the S&P at this time period
    investable_universe = []
    for col in df_investable.columns:
        if ~df_investable[col].iloc[:n].isna().any():
            investable_universe.append(col)
        
    df_investable = df_investable[investable_universe]
    
    return df_investable

In [ ]:
# replace apply_PCA for autoencoder
def apply_PCA(inv, k):
    X = inv.iloc[1:, :]
    pca = PCA(n_components = k) 
    inv_scaled = StandardScaler().fit_transform(X)   
    principal_components = pca.fit_transform(inv_scaled)

    df = pd.DataFrame(data = principal_components)
    
    #For explained variance table
    components = pca.components_
    component_explained_var = pca.explained_variance_ratio_ * 100
    
    comp_names = ['PCA' + str(i) for i in range(1, len(component_explained_var) + 1)]

    pca_results = pd.DataFrame(data = component_explained_var, index = comp_names)
    pca_results.columns = ['Explained variance (%)']
    pca_results['Explained variance (%)'] = pca_results['Explained variance (%)'].round(2)
    
    return df

In [ ]:
def define_y(inv, stock):
    y = inv[[stock]].iloc[:-1]
    
    return y

In [ ]:
def train_test(X, y):
    X_train = X.iloc[1:, :]
    X_test = X.iloc[0:1, :]
    y_train = y.iloc[1:]
    y_test = y.iloc[0:1]
    
    return X_train, y_train, X_test, y_test

In [ ]:
# Converts investable dataframe into 3D tensor for input into RNN fitting
# Eg. if investable_df has shape (52, 635), it is converted to a numpy array of shape (52, 1, 635)

def formatX(investable_df):
    train_values = []
    # Iterates through each day in investable df and appends feature values to train_values
    for i in range(len(investable_df.index)):
        train_values.append(investable_df.iloc[i].values)
    train_values = np.array(train_values) # converts to numpy array
    train_values = np.reshape(train_values, (train_values.shape[0], 1, train_values.shape[1])) # reshapes to 3-dimensional
    return train_values

In [ ]:
# Formats output for a stock as a np array
# Takes in a stock, a date, and a number of rows to look back on and formats the output values as a numpy array
# Eg. formatY(stockX, Feb.25, 80) outputs the returns for stockX starting from 80 days before Feb.25 up to Feb.25

def formatY(y):
    return np.array(y)[:, 0]

In [ ]:
from keras.callbacks import EarlyStopping

def model_fit(X_train, y_train):
    
    # Converts features and output values to a 3D tensor and numpy array respectively
    convertedX = formatX(X_train)
    convertedY = formatY(y_train)
    
    # Build LSTM model with autoencoder
    model = Sequential()
    model.add(LSTM(100,activation='relu', input_shape =(convertedX.shape[1], convertedX.shape[2])))
    model.add(RepeatVector(convertedX.shape[1]))
    model.add(LSTM(100,activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer="adam", loss='mse')#rnn: optimizer='rmsprop'
    early_stopping = EarlyStopping(monitor="val_loss", patience=5)
    
    # Fit model on formatted data
    model.fit(convertedX, convertedY, validation_split=0.2, epochs=50, batch_size=16, verbose=0, callbacks=[early_stopping])
    
    return model

In [ ]:
def model_predict(model, X_test):
    convertedX = formatX(X_test)
    yhat = model.predict(X_test)
    
    return yhat

In [ ]:
def predict_returns(t, n, k, model, refit=True):
    inv = get_investable(t, n)
    X = apply_PCA(inv, k)
    
    returns_t = pd.DataFrame(index = inv.columns, columns = ['Pred', 'Actual'])
    
    for stock in inv.columns:
        y = define_y(inv, stock)
        X_train, y_train, X_test, y_test = train_test(X, y)
        
        if refit:
            model = model_fit(X_train, y_train)
        
        yhat = model_predict(model, X_test)[0][0]
        returns_t['Pred'].loc[stock] = yhat
        returns_t['Actual'].loc[stock] = y_test.values[0][0]
    
    return returns_t, model

In [ ]:
def rank_stocks(returns, num_stocks):
    pred_returns = returns.sort_values(by = 'Pred', ascending = False)
    topn = pred_returns.head(num_stocks)
    botn = pred_returns.tail(num_stocks)
    
    return topn, botn

In [ ]:
def portfolio_return(topn, botn, returns):
    return_t = topn['Actual'].mean() - botn['Actual'].mean()
    
    return return_t

In [ ]:
def pipeline(n, k, num_stocks):

    time_range = returns.loc['2015':'2021'].index
    
    for i in range(len(time_range)):
        if time_range[i] in returns_monthly.index:
            time_range = time_range[i:]
            break
    
    portfolio = pd.DataFrame(index = time_range, columns = ['Portfolio Return'])
    current_model = Sequential()
    
    count = 0
    for t in time_range[:-1]:
        if t in returns_monthly.index:
            pred_actual, current_model = predict_returns(t, n, k, current_model, refit=True)
        else:
            pred_actual, current_model = predict_returns(t, n, k, current_model, refit=False)
        
        topn, botn = rank_stocks(pred_actual, num_stocks)
        return_t = portfolio_return(topn, botn, pred_actual)
        t_index = time_range.get_loc(t) + 1
        portfolio['Portfolio Return'].loc[time_range[t_index]] = return_t
        
        count +=1
        print(f'{(count/len(time_range))*100:.2f}% complete')
    
    portfolio['Portfolio Return'] = portfolio['Portfolio Return'].astype('float')
    
    return portfolio

In [ ]:
#started at XX:XX time
print(pd.datetime.now())

In [ ]:
portfolio = pipeline(50, 20, 5)

In [ ]:
portfolio.dropna(inplace = True)
portfolio

In [ ]:
portfolio.to_csv('results/LSTM_autoencoder.csv')

In [ ]:
import matplotlib.ticker as ticker
import matplotlib.dates as mdates

fig, axes = plt.subplots(figsize=(30,15))
sns.barplot(x = portfolio.index, y = 'Portfolio Return', data = portfolio, color = 'grey')

axes.xaxis.set_major_locator(mdates.YearLocator())
axes.xaxis.set_minor_locator(mdates.MonthLocator())

ticklabels = [item.strftime('%Y') for item in portfolio.resample('Y').mean().index.to_period('Y')]

axes.xaxis.set_major_formatter(ticker.FixedFormatter(ticklabels))

plt.xticks(rotation = 'vertical')
axes.set_title('Portfolio Returns')

sns.set(font_scale=2)
plt.axhline(0)

plt.tight_layout()

plt.show()

In [ ]:
# in original pipeline, this code does not use
portfolio_monthly = portfolio.resample('M').mean()
portfolio_monthly

In [ ]:
avg_return = portfolio['Portfolio Return'].mean()
print(f'Average return is {avg_return:.2f} %')

In [ ]:
rolling_avg = pd.read_csv('results/LSTM_autoencoder.csv', index_col = 'date', parse_dates = True)(252).mean())

In [ ]:
rolling_avg = pd.DataFrame(data = rolling_avg['Portfolio Return'].rolling(252).mean())

In [ ]:
rolling_avg.dropna(inplace=True)

In [ ]:
# 12-Month Rolling Average Portfolio Returns

fig, axes = plt.subplots(figsize=(30,15))
sns.lineplot(x=rolling_avg.index, y="Portfolio Return", data=rolling_avg, color = '#0c8c84ff')

plt.xticks(rotation = 'vertical', fontsize = 25)
plt.yticks(fontsize = 20)

axes.set_title('LSTM with Autoencoder: 12-Month Rolling Average Portfolio Returns', fontsize = 35)
axes.set_xlabel('Year', fontsize = 30)
axes.set_ylabel('Portfolio Return (%)', fontsize = 30)

plt.xlim([rolling_avg.index[0], rolling_avg.index[-1]])
plt.axhline(0, color = '#fcc43cff')